<a href="https://colab.research.google.com/github/ea-analisisdatos/web_scraping_google_colab_bookstoscrape/blob/main/WebScraping_BeautifulSoup_bookstoscrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scrarping con BeautifulSoup

Fuentes de Datos:
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Página Web que debe ser scrapeadas: https://books.toscrape.com/catalogue/category/books_1/index.html


In [1]:
# Instalación de librerías
!pip install bs4

In [7]:
# Instalación de librería para exportar el csv automaticamente al repositorio de Github
!pip install GitPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


## Scrapeando la web de libros con la librería BeautifulSoup

In [6]:
# prompt: Puedes arreglar el código anterior para en la columna Nombre en el CSV no presente caracteres especiales como este: Full Moon over Noahâs Ark: An Odyssey to Mount Ararat and Beyond?

import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de la página a scrapear
url = 'https://books.toscrape.com/index.html'

# Hacer la petición a la página
response = requests.get(url)

# Crear un objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Encontrar la lista de categorías
categorias_sidebar = soup.find('ul', class_='nav nav-list').find_all('li')[1:]

# Crear listas para almacenar los datos
nombres = []
estrellas = []
precios = []
disponibilidad = []
categorias = []

# Iterar sobre cada categoría
for categoria_item in categorias_sidebar:
  # Extraer el nombre de la categoría
  categoria = categoria_item.a.text.strip()

  # Construir la URL de la página de la categoría
  categoria_url = 'https://books.toscrape.com/' + categoria_item.a['href']

  # Hacer la petición a la página de la categoría
  categoria_response = requests.get(categoria_url)
  categoria_soup = BeautifulSoup(categoria_response.text, 'html.parser')

  # Encontrar todos los artículos de libros en la página de la categoría
  libros = categoria_soup.find_all('article', class_='product_pod')

  # Iterar sobre cada libro en la categoría
  for libro in libros:
    # Extraer el nombre del libro y limpiar caracteres especiales
    nombre = libro.h3.a['title']
    nombre_limpio = ''.join(c for c in nombre if c.isalnum() or c.isspace())
    nombres.append(nombre_limpio)

    # Extraer las estrellas
    estrella = libro.p['class'][1]
    estrellas.append(estrella)

    # Extraer el precio
    precio = libro.find('p', class_='price_color').text[2:]
    precios.append(precio)

    # Extraer la disponibilidad
    disponible = libro.find('p', class_='instock availability').text.strip()
    disponibilidad.append(disponible)

    # Agregar la categoría al libro
    categorias.append(categoria)

# Crear un DataFrame de Pandas
df = pd.DataFrame({
  'Nombre': nombres,
  'Estrellas': estrellas,
  'Precio': precios,
  'Disponibilidad': disponibilidad,
  'Categoría': categorias
})

# Guardar el DataFrame en un archivo CSV
df.to_csv('libros.csv', index=False)


## Exportando el archio libros.csv hacía el repositorio de Github

In [ ]:
from git import Repo

# Repositorio de GitHub (reemplaza con tu repositorio)
repo_url = 'https://github.com/ea-analisisdatos/web_scraping_google_colab_bookstoscrape.git'

# Directorio local donde clonar el repositorio (si no existe, se creará)
repo_dir = 'mi_repositorio'

# Clonar el repositorio si no existe
if not os.path.exists(repo_dir):
  Repo.clone_from(repo_url, repo_dir)

# Acceder al repositorio
repo = Repo(repo_dir)

# Agregar el archivo CSV al repositorio
repo.index.add(['libros.csv'])

# Hacer commit de los cambios
repo.index.commit('Agregar archivo libros.csv')

# Subir los cambios a GitHub
origin = repo.remote(name='origin')
origin.push()

print('Archivo libros.csv subido a GitHub.')
